In [1]:
import numpy as np
from torch.utils.data import DataLoader
from functions import *
from torch.nn import GRU,Conv1d,Linear,ReLU,Sequential,MSELoss,L1Loss,LeakyReLU,Dropout
from torch.optim import Adam
from pytorch_util import *
from pytorch_models import ConvDropout2dGLU,GRU_NCL

In [2]:
long_page = np.load('../Data/long_page.npy')
long_value = np.load('../Data/long_value.npy')
start_index = np.load('../Data/start_index.npy')
stds = np.load('../Data/stds_log.npy')
means = np.load('../Data/means_log.npy')
long_normal_value = np.load('../Data/long_lognormal_value.npy')

In [3]:
# to account for look back period 365 + 60
start_index = start_index + 430

In [4]:
## to change ##
batch_size = 64
clip = .5

In [5]:
# baseline model
# yhat = long_normal_value[:,803-61:803-60]
# y = long_normal_value[:,803-60:]
# SMAPE_np(y,yhat)
# >> 106.31
# yhat = long_normal_value[:,803-61:803-60]
# y = long_normal_value[:,803-60:]
# np.mean(np.square(y - yhat))
# >>> 1.1114173

In [6]:
# set up data loader
train_gen = SequenceGen(long_value,long_normal_value,start_index,stds,means,useMean=True)
train_gen = DataLoader(train_gen,batch_size,True,num_workers=3,drop_last=True)

val_gen = SequenceGen(long_value,long_normal_value,start_index,stds,means,random=False,useMean=True)
val_gen = DataLoader(val_gen,batch_size,False,num_workers=3)

train_gen_nor = SequenceGenNormalized(long_normal_value,start_index)
train_gen_nor = DataLoader(train_gen_nor,batch_size,True,num_workers=3,drop_last=True)

val_gen_nor = SequenceGenNormalized(long_normal_value,start_index,random=False)
val_gen_nor = DataLoader(val_gen_nor,batch_size)

In [7]:
conv = Sequential(ConvDropout2dGLU(3,8,3),
                  ConvDropout2dGLU(8,16,3),
                  ConvDropout2dGLU(16,32,3),
                  ConvDropout2dGLU(32,64,3))

In [8]:
rnn = GRU_NCL(64,64,2,batch_first=True,dropout=0.5,bidirectional=True)

In [9]:
seq_model = Sequential(conv,rnn)

In [10]:
linear = Sequential(Linear(128,64),LeakyReLU(0.1,True),Dropout(0.5),Linear(64,1))

In [11]:
convert = Sequential(Linear(112,60),LeakyReLU(0.1,True),Dropout(0.5))

In [12]:
# model trained on normalized data
model_nor = CNN_RNN2seq_normalized_NCL(seq_model,linear,convert).to('cuda:0')

In [13]:
loss_func_nor = loss_func_generator_normalized(MSELoss())
# loss_func_nor = loss_func_generator_normalized(L1Loss())
#loss_func_nor = loss_func_generator_normalized(SMAPE)

In [14]:
opt = Adam(trainable_parameter(model_nor),lr=1e-3)

In [15]:
model_nor = fit(5, model_nor, loss_func_nor, opt, train_gen_nor, val_gen_nor,clip,patience=3)

epoch:0, train_loss:0.8234066663010751, val_loss:0.828204333782196
epoch:1, train_loss:0.776049881494107, val_loss:0.8129386305809021
epoch:2, train_loss:0.7656428457696203, val_loss:0.8142498731613159
epoch:3, train_loss:0.757975368024154, val_loss:0.8186010718345642
epoch:4, train_loss:0.752923075986032, val_loss:0.8160735368728638
Training completed in 224.22870445251465s


In [16]:
model = CNN_RNN2seq_logNormalized_NCL(seq_model,linear,convert).to('cuda:0')

In [17]:
opt = Adam(trainable_parameter(model),lr=1e-4)

In [18]:
loss_func = loss_func_generator(SMAPE)

In [19]:
model = fit(5, model, loss_func, opt, train_gen, val_gen,clip)

epoch:0, train_loss:42.955875710996324, val_loss:40.52982711791992
epoch:1, train_loss:40.4137877192522, val_loss:38.55618667602539
epoch:2, train_loss:39.684311942856546, val_loss:38.38835525512695
epoch:3, train_loss:39.29715719371262, val_loss:38.0429801940918
epoch:4, train_loss:38.999867041988075, val_loss:37.69680404663086
Training completed in 265.99101400375366s


In [20]:
model = fit(5, model, loss_func, opt, train_gen, val_gen,clip,lossBest=37.69680404663086)

epoch:0, train_loss:38.70683824608363, val_loss:37.43674087524414
epoch:1, train_loss:38.34681469517051, val_loss:37.09122085571289
epoch:2, train_loss:38.172174513031166, val_loss:37.15676498413086
epoch:3, train_loss:38.02480814839892, val_loss:36.928001403808594
epoch:4, train_loss:37.95617251618538, val_loss:36.95585250854492
Training completed in 264.7246217727661s


In [21]:
model = fit(10, model, loss_func, opt, train_gen, val_gen,clip,lossBest=36.95585250854492)

epoch:0, train_loss:37.82309138540159, val_loss:37.09172058105469
epoch:1, train_loss:37.775164135750096, val_loss:36.836612701416016
epoch:2, train_loss:37.726277976950215, val_loss:36.65525817871094
epoch:3, train_loss:37.60961759102777, val_loss:36.988887786865234
epoch:4, train_loss:37.51320474061323, val_loss:36.57807159423828
epoch:5, train_loss:37.474057098744446, val_loss:36.61368942260742
epoch:6, train_loss:37.3048846447406, val_loss:36.478267669677734
epoch:7, train_loss:37.354965675433064, val_loss:36.84625244140625
epoch:8, train_loss:37.33437985830356, val_loss:36.76797103881836
epoch:9, train_loss:37.34441913644267, val_loss:36.79045486450195
Training completed in 531.3601024150848s


In [22]:
model = fit(10, model, loss_func, opt, train_gen, val_gen,clip,lossBest=36.478267669677734)

epoch:0, train_loss:37.373776925042506, val_loss:36.36301803588867
epoch:1, train_loss:37.37110090502803, val_loss:36.606746673583984
epoch:2, train_loss:37.29046982187064, val_loss:36.35445785522461
epoch:3, train_loss:37.26422443291066, val_loss:36.48810577392578
epoch:4, train_loss:37.23846299364159, val_loss:36.62095642089844
epoch:5, train_loss:37.22501837676053, val_loss:36.30223846435547
epoch:6, train_loss:37.204463266955756, val_loss:36.582359313964844
epoch:7, train_loss:37.140206327092464, val_loss:36.51466369628906
epoch:8, train_loss:37.075245787072056, val_loss:36.46876525878906
epoch:9, train_loss:37.1069020562839, val_loss:36.24162292480469
Training completed in 536.5678734779358s
